In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import StrOutputParser
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser, JSONAgentOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents import AgentExecutor
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain import hub
from langchain.tools.render import render_text_description_and_args

from utils import map_openbb_collection_to_langchain_tools

In [10]:
# Let's set-up our tool retrieval

fundamental_openbb_tools = map_openbb_collection_to_langchain_tools("/equity/fundamental")

docs = [
    Document(page_content=t.description, metadata={"index": i})
    for i, t in enumerate(fundamental_openbb_tools)
]

vector_store = FAISS.from_documents(docs, OpenAIEmbeddings())

In [11]:
retriever = vector_store.as_retriever(search_kwargs={'k': 2}) # <- now returns top 2

def get_tools(query):
    docs = retriever.get_relevant_documents(query)
    return [fundamental_openbb_tools[d.metadata["index"]] for d in docs]

# Quick test
fetched_tools = get_tools("income statement")
len(fetched_tools)

2

In [18]:
# Let's make it easy to create react agents, since we'll need a lot of them later.

def langchain_react_agent(tools):
    """Define a ReAct agent bound with specific tools."""
    prompt = hub.pull("hwchase17/react-multi-input-json")
    prompt = prompt.partial(
        tools=render_text_description_and_args(tools),
        tool_names=", ".join([t.name for t in tools])
    )

    llm = ChatOpenAI(model="gpt-4-1106-preview").bind(stop=["\nObservation"])

    chain = (
        {
            "input": lambda x: x["input"],
            "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"])
        }
        | prompt
        | llm
        | JSONAgentOutputParser()
    )

    agent_executor = AgentExecutor(agent=chain, tools=tools, verbose=False, return_intermediate_steps=False, handle_parsing_errors=True)
    return agent_executor

agent_executor=langchain_react_agent(tools=fetched_tools)
result = agent_executor.invoke({"input": input})

In [27]:
# The primary goal is to 
# 1. Break a larger question down into subquestions + the appropriate query to fetch the right tool to answer the subquestion
# 2. Retrieve the right tools for each subquestion
# 3. Answer each subquestion using a ReAct agent
# 4. To combine all of the subquestion answers to generate a final answer.

# Part 1 break it into subquestions
from langchain.schema import BaseOutputParser
import json

class SimpleJsonListOutputParser(BaseOutputParser):
    def parse(self, text: str) -> list[dict]:
        cleaned_text = text.strip()
        try:
            content = json.loads(cleaned_text)
            return content
        except json.JSONDecodeError as e:
            raise e

system_message = """\
You are a world class state of the art agent.

You have access to multiple tools, via a "fetch_tools" function that will retrieve the tools you need.
Each retrieved tool represents a different data source or API.


Your purpose is to help answer a complex user question by generating a list of sub
questions, which act as natural language queries to the "fetch_tools" function to retrieve the relevant tools.

These are the guidelines you consider when completing your task:
* Be as specific as possible
* The sub questions should be relevant to the user question
* The sub questions should be answerable by the retrieved tools provided
* You can generate multiple sub questions
* You don't need to query for a tool if you don't think it's relevant

Provide your queries in the following format:
```json
[
    {{
        "subquestion": "<the subquestion>",
         "query": "<the query argument for the fetch_tool function>"
    }},
    ... REPEAT AS MANY TIMES AS NECESSARY FOR EACH SUBQUESTION.
]
```

For example:
```json
[
    {{
        "subquestion": "How much debt does TSLA have?",
         "query": "debt"
    }},
    {{
        "subquestion": "How much cash does TSLA have?",
         "query": "cash"
    }}
]
```

Provide your answer in the above format ONLY. Do not provide any other output.
"""

human_message = """\
    ## User Question
    {input}
    """

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        ("human", human_message),
    ]
)

llm = ChatOpenAI(model="gpt-4") . # gpt-3.5-turbo works well, but gpt-4-1106-preview isn't good at returning JSON.

subquestion_chain = (
    {
        "input": lambda x: x["input"]
    }
    | prompt
    | llm
    | SimpleJsonListOutputParser()
)

# Our high-level question we're going to attempt to answer
INPUT = "Perform a fundamentals financial analysis of AMZN using the most recently available data. What do you find that's interesting?"

subquestions = subquestion_chain.invoke({"input": INPUT})

for subquestion in subquestions:
    print(subquestion["subquestion"], " :: tool query: " + subquestion["query"])

What is the current stock price of AMZN?  :: tool query: stock_price
What is the current market capitalization of AMZN?  :: tool query: market_cap
What is the P/E ratio of AMZN?  :: tool query: pe_ratio
What is the earnings per share (EPS) of AMZN?  :: tool query: eps
What is the revenue of AMZN?  :: tool query: revenue
What is the net income of AMZN?  :: tool query: net_income
What is the debt of AMZN?  :: tool query: debt
What is the cash reserve of AMZN?  :: tool query: cash
What is the return on equity (ROE) of AMZN?  :: tool query: roe
What is the return on assets (ROA) of AMZN?  :: tool query: roa
What is the profit margin of AMZN?  :: tool query: profit_margin


In [28]:
# Part 2 is to fetch the appropriate tool for each subquestion
# (We'll be sneaky and re-use the datastructure)
for subquestion in subquestions:
    tools = get_tools(subquestion['query'])
    subquestion['tools'] = tools

In [29]:
# Part 3 is to answer each of the subqueries. We'll use a ReAct agent to do this.
from langchain.schema.runnable import RunnableParallel

agents = []
for i, subquestion in enumerate(subquestions):
    react_agent = langchain_react_agent(tools=subquestion['tools'])
    agents.append(react_agent)

In [30]:
# Run the agents to answer the subquestions
for i, subquestion in enumerate(subquestions):
    input = f"""\
    Given the following high-level question: {INPUT}
    Answer the following subquestion: {subquestion['subquestion']}

    Give the answer in clear, understandable english in a bulleted list.
    Explain your reasoning, and make reference to the data.
    Do not format the final answer string using JSON.

    For example:
    - Observation One
    - Observation Two
    - Observation Three
    """
    try:
        result = agents[i].invoke({"input": input})
        output = result["output"]
    except Exception:  # Terrible practice, but it'll do for now.
        output = "I was unable to answer the subquestion using the available tool."   # We'll include the error message in the future

    print(subquestion["subquestion"])
    print("----")
    print(output)
    print("=======")
    
    # We'll misbehave and re-use the same datastructure again
    subquestion["observation"] = output

What is the current stock price of AMZN?
----
- The current stock price of AMZN (Amazon.com, Inc.) is $146.71.
What is the current market capitalization of AMZN?
----
- The current market capitalization of Amazon.com, Inc. (AMZN) is approximately $1.516 trillion.
What is the P/E ratio of AMZN?
----
- The P/E ratio of AMZN is approximately -311.11.
What is the earnings per share (EPS) of AMZN?
----
- The most recent available actual EPS for AMZN is $0.94 for the period ending September 30, 2023.
- The EPS was estimated to be $0.58, indicating that AMZN outperformed the earnings expectation for that period.
- It's also noted that the actual EPS has shown some variability over the recent periods, with the previous quarter being $0.65 against an estimate of $0.35.
What is the revenue of AMZN?
----
According to the most recent data available for the fiscal year ending December 31, 2022, Amazon.com, Inc. (AMZN) reported the following revenue figures across its various business lines:
- Onlin

In [31]:
# Part 4 is to generate a response based on the answers to each of the subquestions
system_message = """\
    Given the following high-level question: 

    Question: {input}

    And the following subquestions and subsequent observations:

    {subquestions}

    Answer the high-level question. Give your answer in a bulleted list.
    """

def render_subquestions_and_answers(subquestions):
    output = ""
    for subquestion in subquestions:
        output += "Subquestion: " +  subquestion["subquestion"] + "\n"
        output += "Observation: " +  subquestion["observation"] + "\n\n"

    return output

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message)
    ]
)

llm = ChatOpenAI(model="gpt-4-1106-preview")

final_chain = (
    {
        "input": lambda x: x["input"],
        "subquestions": lambda x: render_subquestions_and_answers(x["subquestions"])
    }
    | prompt
    | llm
)

result = final_chain.invoke({"input": INPUT, "subquestions": subquestions})

In [195]:
print(result.content)  # Et voila

Based on the most recently available data, a fundamentals financial analysis of Amazon (AMZN) reveals the following key points:

- Revenue: Amazon has a diversified revenue stream with significant contributions from various sectors:
  - Online Stores: $220,004,000,000
  - Amazon Web Services (AWS): $80,096,000,000
  - Third-Party Seller Services: $117,716,000,000
  - Subscription Services: $35,218,000,000
  - Advertising: $37,739,000,000
  - Physical Stores: $18,963,000,000
  - Other Services: $4,247,000,000

- Net Income: Amazon reported a net loss of -$2,722,000,000 for the fiscal year ending December 31, 2022, which is notable given the company's scale and history of profitability.

- Operating Cash Flow: Despite the net loss, Amazon generated a strong operating cash flow of $46.752 billion, indicating that the company's core operations remain cash-generative.

- Current Ratio: With a current ratio of approximately 0.94, Amazon may have slightly more short-term liabilities than shor